In [1]:
import numpy as np
import random
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
def aug_random(train,num_aug):
    print ("random")
    new = []
    for i in range(num_aug):
        index = random.sample(list(np.arange(len(train))), 2)
    #     print index
        new_sample = 0.5*train[index[0], :, :, :] + 0.5*train[index[1], :, :, :]
        new.append(new_sample)
    new = np.array(new)
    return new

In [3]:
def aug_appliance(train, num_aug):
    print ("appliance")
    new = np.zeros((num_aug, 6, 112, 24))
    for i in range(num_aug):
        home_agg = np.zeros((112,24))
        for appliance in range(1,6):
            index = np.random.choice(list(range(len(train))))
            new[i, appliance, :, :] = train.copy()[index, appliance, : :]
            home_agg += train.copy()[index, appliance, :, :]
        new[i, 0, :, :] = home_agg
    return new

In [4]:
def aug_noise(train, num_aug):
    print ("noise")
    new = []
    for i in range(num_aug):
        index = np.random.choice(list(range(len(train))))
        noise = np.random.normal(0,1,112*24*5).reshape(5, 112, 24)
        new_sample = train.copy()[index]
        new_sample[1:] = new_sample[1:] + noise
        new_sample[0] = 0 
        for j in range(1, 6):
            new_sample[0] += new_sample[j]
        new.append(new_sample)
    new = np.array(new)
    return new

In [5]:
def selective(train, test):
    test_aggregate = test[:, 0, :, :]
    train_aggregate = train[:, 0, :, :]
    
    test_aggregate = test_aggregate.reshape(test_aggregate.shape[0], -1)
    train_aggregate = train_aggregate.reshape(train_aggregate.shape[0], -1)
    
    concated = np.vstack([train_aggregate, test_aggregate])
    
    similarity = cosine_similarity(concated)
    similarity = similarity[:len(train_aggregate), len(train_aggregate):]
    
    train_max = similarity.max(axis=1)
    
    k = int(0.5*len(train_aggregate))
    index = np.argpartition(train_max, -k)[-k:]
    
    return train[index]

In [18]:
def augmented_data(train, num_aug, case, test = None, select=True):
    
    if num_aug == 0:
        return train
    
    if select:
        selected_train = selective(train, test)

        
    if case == 1:
        new = aug_random(selected_train, num_aug)
    if case == 2:
        new = aug_appliance(selected_train, num_aug)
    if case == 3:
        new = aug_noise(selected_train, num_aug)
    print (new.shape)
    print (train.shape)
    return np.vstack([train, new])


In [19]:
train, test = get_train_test(1, 5, 0)

In [20]:
new_train = augmented_data(train, 2, 1, test, True)

random
(2, 6, 112, 24)
(54, 6, 112, 24)


In [21]:
train.shape

(54, 6, 112, 24)

In [22]:
new_train.shape

(56, 6, 112, 24)